# K8s Provider and Universe

### Setup

First, import the required packages.

Next, specify some important variables:
* `platform_address`: The address of the Yugabyte Platform API
* `platform_api_key`: The API key used to authenticate with the Platform API

Finally, open a HTTP connection to the Yugabyte Platform API.

In [44]:
import os
import http.client
import json
from pprint import pprint

platform_address = os.getenv('API_BASE_URL')
platform_api_key = os.getenv('YB_API_KEY')

conn = http.client.HTTPConnection(f"{platform_address}")

headers = {
  'Content-Type': "application/json",
  'X-AUTH-YW-API-TOKEN': f"{platform_api_key}"
}


### Get Session Info

Make an API call to session endpoint to determine customer UUID.

In [41]:
conn.request("GET", "/api/v1/session_info", headers=headers)

res = conn.getresponse()
data = res.read()
session_info = json.loads(data)
customer_uuid = session_info['customerUUID']

print('Customer UUID:\n%s' % customer_uuid)

Customer UUID:
22c55e88-813e-4f4e-9b44-c1f99e6bc798


### Get Provider UUID By Name

Make an API call to the provider endpoint to determine provider UUID and regions.

In [42]:
provider_name = 'gke-multi-zone' # my-gcp-rovider 
url = f"/api/v1/customers/{customer_uuid}/providers"
conn.request("GET", url, headers=headers)
res = conn.getresponse()

provider_list = json.load(res)
for provider in provider_list:
    if provider['name'] == provider_name and provider['active'] == True:
        provider_uuid = provider['uuid']
        region_list = [region['uuid'] for region in provider['regions']]
        break

print('Provider UUID:\n%s' % provider_uuid)
print('Regions:\n[%s]' % ', '.join(region_list))

Provider UUID:
1f65440b-9cb3-482b-9f1c-c076f6e62f81
Regions:
[f47d55a9-74bd-4b80-97d3-ca3cecbe07fc]


### Get Access Key

Make an API call to the access key endpoint to determine access key for provider.

In [43]:
url = f"/api/v1/customers/{customer_uuid}/providers/{provider_uuid}/access_keys"
conn.request("GET", url, headers=headers)
res = conn.getresponse()

access_key_list = json.load(res)
access_key_code = access_key_list[0]['idKey']['keyCode'] if access_key_list else "yugabyte-default"
print('Access Key:\n%s' % access_key_code)

Access Key:
yugabyte-default


### Get Instance Types Available

Make an API call to see the instance types available for the provider.

In [27]:
url = f"/api/v1/customers/{customer_uuid}/providers/{provider_uuid}/instance_types"
conn.request("GET", url, headers=headers)
res = conn.getresponse()

instance_type = None
for i in json.load(res):
    print(i['instanceTypeCode'])
    if i['instanceTypeCode'] in ['large']: # desired aws or gcp type code
        print('^^^^^^^ Found !!!!!!!!!!!')
        instance_type = i
        break

pprint(instance_type['instanceTypeDetails'])

small
xsmall
large
^^^^^^^ Found !!!!!!!!!!!
{'volumeDetailsList': [{'mountPath': '/mnt/d0',
                        'volumeSizeGB': 100,
                        'volumeType': 'SSD'}]}


In [28]:
new_universe = {
    'clusters': [
        {
            'clusterType': 'PRIMARY',
            'userIntent': {
                'universeName': 'my-k8s-universe', # or my-gcp-universe
                'providerType': 'kubernetes',
                'provider': provider_uuid,
                'regionList': region_list,
                'numNodes': 3,
                'replicationFactor': 3,
                'instanceType': instance_type['instanceTypeCode'],
                'deviceInfo': {
                    'numVolumes': 1,
                    'volumeSize': instance_type['instanceTypeDetails']['volumeDetailsList'][0]['volumeSizeGB'],
                    'storageClass': "standard"
                },
                'assignPublicIP': True,
                'useTimeSync': True,
                'enableYSQL': True,
                'enableYEDIS': False,
                'enableNodeToNodeEncrypt': True,
                'enableClientToNodeEncrypt': True,
                'enableVolumeEncryption': False,
                'ybSoftwareVersion': '2.7.3.0-b80',
                'accessKeyCode': access_key_code,
                'tserverGFlags': {},
                'masterGFlags': {},
            }
        },
    ],
}

### Create Universe

Make API call to create new universe.

In [29]:
url = f"/api/v1/customers/{customer_uuid}/universes/clusters"
conn.request("POST", url, json.dumps(new_universe), headers)
res = conn.getresponse()
pprint(json.load(res))

{'resourceUUID': 'b94885cc-75a9-4a54-893c-9db844e27252',
 'taskUUID': '8de55be2-69bf-4813-b656-f10322adf8d1'}
